In [1]:
library(data.table)
library(parallel)

In [5]:
read_bin_packing_instance <- function(filename) {
  lines <- readLines(filename)
  P <- as.integer(lines[1])  # Número de instancias de problemas
  problems <- list()
  index <- 2
  for (i in 1:P) {
    problem_id <- trimws(lines[index])
    index <- index + 1
    bin_info <- strsplit(trimws(lines[index]), "\\s+")[[1]]
    bin_capacity <- as.numeric(bin_info[1])
    num_items <- as.integer(bin_info[2])
    best_known_solution <- as.integer(bin_info[3])
    index <- index + 1
    items <- numeric(num_items)
    for (j in 1:num_items) {
      items[j] <- as.numeric(trimws(lines[index]))
      index <- index + 1
    }
    problems[[i]] <- list(
      problem_id = problem_id,
      bin_capacity = bin_capacity,
      num_items = num_items,
      best_known_solution = best_known_solution,
      items = items
    )
  }
  return(problems)
}

In [7]:
# Función para transformar la solución en una lista de contenedores
transform_solution <- function(solution, item_sizes, bin_capacity) {
  bins <- list()
  current_bin <- numeric()
  current_bin_capacity <- 0
  
  for (i in solution) {
    item_size <- item_sizes[i]
    if (current_bin_capacity + item_size <= bin_capacity) {
      current_bin <- c(current_bin, item_size)
      current_bin_capacity <- current_bin_capacity + item_size
    } else {
      bins <- c(bins, list(current_bin))
      current_bin <- item_size
      current_bin_capacity <- item_size
    }
  }
  bins <- c(bins, list(current_bin))
  return(bins)
}

In [47]:
shaking <- function(solution, k) {
  n <- length(solution)
  perturbation_type <- sample(1:2, 1)
  
  if (perturbation_type == 1) {
    # Realizar k intercambios aleatorios
    for (i in 1:k) {
      idx1 <- sample(1:n, 1)
      idx2 <- sample(1:n, 1)
      temp <- solution[idx1]
      solution[idx1] <- solution[idx2]
      solution[idx2] <- temp
    }
  } else if (perturbation_type == 2) {
    # Perturbación basada en segmentos
    segment_length <- sample(2:floor(n / 2), 1)
    start_idx <- sample(1:(n - segment_length), 1)
    segment <- solution[start_idx:(start_idx + segment_length - 1)]
    shuffled_segment <- sample(segment)
    solution[start_idx:(start_idx + segment_length - 1)] <- shuffled_segment
  }
  
  return(solution)
}


In [6]:
best_fit_decreasing <- function(elements, container_capacity) {
  original_indices <- seq_along(elements)
  sorted_indices <- order(elements, decreasing = TRUE)
  elements_sorted <- elements[sorted_indices]
  
  containers <- list()
  assignments <- integer(length(elements)) # Para almacenar las asignaciones de contenedores
  
  for (i in seq_along(elements_sorted)) {
    element <- elements_sorted[i]
    best_fit_index <- -1
    min_space_left <- container_capacity + 1
    
    for (j in seq_along(containers)) {
      space_left <- container_capacity - sum(unlist(containers[[j]]))
      if (space_left >= element && space_left < min_space_left) {
        min_space_left <- space_left
        best_fit_index <- j
      }
    }
    
    if (best_fit_index == -1) {
      containers <- append(containers, list(c(element)))
      best_fit_index <- length(containers)
    } else {
      containers[[best_fit_index]] <- c(containers[[best_fit_index]], element)
    }
    
    assignments[sorted_indices[i]] <- best_fit_index
  }
  
  return(assignments)
}


In [35]:
local_search_two <- function(bins, bin_capacity, binpacking_instance, max_iter = 10, max_exchanges = 50, tabu_list = list(), tabu_size = 10) {
  cat("Starting local_search", "\n")
  flush.console()
  current_solution <- bins
  best_solution <- current_solution
  best_cost <- binpacking_instance$evaluate(current_solution)
  improved <- TRUE
  iteration <- 0
  no_improve_iter <- 0

  while (improved && iteration < max_iter) {
    improved <- FALSE
    iteration <- iteration + 1
    cat("Iteration:", iteration, "\n")
    
    # Generar un vecindario reducido aleatorio
    exchanges <- expand.grid(1:(length(current_solution) - 1), (2):length(current_solution))
    exchanges <- exchanges[sample(nrow(exchanges), min(nrow(exchanges), max_exchanges)), ]
    
    for (k in 1:nrow(exchanges)) {
      i <- exchanges[k, 1]
      j <- exchanges[k, 2]
      
      new_solution <- current_solution
      new_solution[c(i, j)] <- new_solution[c(j, i)]
      
      new_cost <- binpacking_instance$evaluate(new_solution)
      
      
      if (new_cost < best_cost && !any(sapply(tabu_list, function(x) all(x == new_solution)))) {
        best_solution <- new_solution
        best_cost <- new_cost
        improved <- TRUE
        no_improve_iter <- 0
        tabu_list <- c(tabu_list, list(new_solution))  # Añadir la nueva solución a la lista Tabú
        if (length(tabu_list) > tabu_size) tabu_list <- tabu_list[-1]  # Mantener un tamaño fijo para la lista Tabú
        cat("Improved: new best_cost =", best_cost, "\n")
        flush.console()
        break  # Salir del bucle interno si se encuentra una mejora
      }
    }
    
    if (!improved) {
      no_improve_iter <- no_improve_iter + 1
      cat("No improvement found in iteration", iteration, "\n")
      flush.console()
    }
  }
  
  cat("Finished local_search with best_cost =", best_cost, "\n")
  flush.console()
  return(best_solution)
}

In [40]:
local_search <- function(bins, bin_capacity, binpacking_instance, max_iter = 10, max_exchanges = 50, tabu_list = list(), tabu_size = 10) {
  current_solution <- bins
  best_solution <- current_solution
  best_cost <- binpacking_instance$evaluate(current_solution)
  improved <- TRUE
  iteration <- 0

  while (improved && iteration < max_iter) {
    improved <- FALSE
    iteration <- iteration + 1
    
    # Generar un vecindario reducido aleatorio
    exchanges <- expand.grid(1:(length(current_solution) - 1), (2):length(current_solution))
    exchanges <- exchanges[sample(nrow(exchanges), min(nrow(exchanges), max_exchanges)), ]
    
    for (k in 1:nrow(exchanges)) {
      i <- exchanges[k, 1]
      j <- exchanges[k, 2]
      
      new_solution <- current_solution
      new_solution[c(i, j)] <- new_solution[c(j, i)]
      
      new_cost <- binpacking_instance$evaluate(new_solution)
      
      if (new_cost < best_cost && !any(sapply(tabu_list, function(x) all(x == new_solution)))) {
        best_solution <- new_solution
        best_cost <- new_cost
        improved <- TRUE
        tabu_list <- c(tabu_list, list(new_solution))  # Añadir la nueva solución a la lista Tabú
        if (length(tabu_list) > tabu_size) tabu_list <- tabu_list[-1]  # Mantener un tamaño fijo para la lista Tabú
        break  # Salir del bucle interno si se encuentra una mejora
      }
    }
    
    if (improved) {
      current_solution <- best_solution
    }
  }
  

  return(best_solution)
}

In [48]:
library(digest)

In [49]:
BinPacking <- setRefClass(
  "BinPacking",
  fields = list(
    item_sizes_dt = "data.table",
    bin_capacity = "numeric",
    eval_cache = "list"
  ),
  methods = list(
    evaluate = function(permutation) {
      perm_key <- digest(permutation, algo = "md5")  # Usar digest para una clave de caché eficiente
      if (!is.null(eval_cache[[perm_key]])) {
        return(eval_cache[[perm_key]])
      }
      
      dt <- item_sizes_dt[permutation, ]
      item_sizes <- dt$size
      
      bin_capacities <- numeric(length(item_sizes))  # Preasignar un vector de tamaño máximo posible
      current_bin_capacity <- 0
      bin_count <- 1  # Iniciar el conteo de bins con 1
      
      for (item_size in item_sizes) {
        if (current_bin_capacity + item_size <= bin_capacity) {
          current_bin_capacity <- current_bin_capacity + item_size
        } else {
          bin_capacities[bin_count] <- current_bin_capacity
          current_bin_capacity <- item_size
          bin_count <- bin_count + 1
        }
      }
      
      bin_capacities[bin_count] <- current_bin_capacity
      
      eval_cache[[perm_key]] <<- bin_count  # Guardar el número de bins en la caché
      
      return(bin_count)
    },
    initialize = function(item_sizes, bin_capacity) {
      .self$item_sizes_dt <- data.table(index = 1:length(item_sizes), size = item_sizes)
      .self$bin_capacity <- bin_capacity
      .self$eval_cache <- list()
    }
  )
)


In [51]:
vns <- function(binpacking_instance, max_iter = 100, max_no_improve = 10) {
  initial_solution <- best_fit_decreasing(binpacking_instance$item_sizes_dt$size, binpacking_instance$bin_capacity)
  cat("initial solution", initial_solution, "\n")
  best_solution <- initial_solution
  best_cost <- binpacking_instance$evaluate(initial_solution)
  no_improve <- 0
  
  cost_per_iteration <- numeric()  # Cambiar a una lista dinámica
  
  for (iter in 1:max_iter) {    
    k <- 1
    while (k <= max_no_improve) {
      new_solution <- shaking(best_solution, k)
      new_solution <- local_search(new_solution, binpacking_instance$bin_capacity, binpacking_instance)
      new_cost <- binpacking_instance$evaluate(new_solution)
      
      cost_per_iteration <- c(cost_per_iteration, new_cost)  # Guardar new_cost
      
      if (new_cost < best_cost) {
        best_solution <- new_solution
        best_cost <- new_cost
        no_improve <- 0
        k <- 1
      } else {
        k <- k + 1
        no_improve <- no_improve + 1
      } 
      if (no_improve >= max_no_improve) break
    }
  }
  
  return(list(solution = best_solution, cost_per_iteration = cost_per_iteration))
}


In [ ]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
instance

In [ ]:
binpacking_instance <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

execution_time <- system.time({
  result <- vns(binpacking_instance, 100, 5)
}) 

best_solution <- result$solution
cost_per_iteration <- result$cost_per_iteration

# Mostrar resultados
cat("Execution Time:", execution_time, "\n")
print(best_solution)  
print(paste("Number of bins:", binpacking_instance$evaluate(best_solution)))

par(bg = "white")
# Graficar la curva de convergencia
plot(cost_per_iteration, type = 'l', main = "Curva de Convergencia", xlab = "Iteracion", ylab = "Costo", col = "black")
beepr::beep()

In [ ]:
file_path <- "./instances/binpack5.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[4]]
instance

In [ ]:
binpacking_instance <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

execution_time <- system.time({
  result <- vns(binpacking_instance, 1000, 5)
}) 

best_solution <- result$solution
cost_per_iteration <- result$cost_per_iteration

# Mostrar resultados
cat("Execution Time:", execution_time, "\n")
print(best_solution)  
print(paste("Number of bins:", binpacking_instance$evaluate(best_solution)))

par(bg = "white")
# Graficar la curva de convergencia
plot(cost_per_iteration, type = 'l', main = "Curva de Convergencia", xlab = "Iteracion", ylab = "Fitness", col = "black")
beepr::beep() 

In [52]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
instance

Warning message in readLines(filename):
"incomplete final line found on './instances/binpack1.txt'"


$problem_id
[1] "u120_00"

$bin_capacity
[1] 150

$num_items
[1] 120

$best_known_solution
[1] 48

$items
  [1] 42 69 67 57 93 90 38 36 45 42 33 79 27 57 44 84 86 92 46 38 85 33 82 73 49
 [26] 70 59 23 57 72 74 69 33 42 28 46 30 64 29 74 41 49 55 98 80 32 25 38 82 30
 [51] 35 39 57 84 62 50 55 27 30 36 20 78 47 26 45 41 58 98 91 96 73 84 37 93 91
 [76] 43 73 85 81 79 71 80 76 83 41 78 70 23 42 87 43 84 60 55 49 78 73 62 36 44
[101] 94 69 32 96 70 84 58 78 25 80 58 66 83 24 98 60 42 43 43 39

In [54]:
Rprof("vns_profile.out")

binpacking_instance <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

execution_time <- system.time({
  result <- vns(binpacking_instance, 2000, 10)
}) 

best_solution <- result$solution
cost_per_iteration <- result$cost_per_iteration

# Detener perfilado
Rprof(NULL)

# Analizar resultados del perfilado
summaryRprof("vns_profile.out")


# Mostrar resultados
cat("Execution Time:", execution_time, "\n")
print(best_solution)  
print(paste("Number of bins:", binpacking_instance$evaluate(best_solution)))

par(bg = "white")
# Graficar la curva de convergencia
plot(cost_per_iteration, type = 'l', main = "Curva de Convergencia", xlab = "Iteracion", ylab = "Fitness", col = "black")
beepr::beep() 


initial solution 41 26 22 7 7 12 45 45 34 42 39 30 34 8 39 17 14 9 24 45 15 47 24 37 2 27 10 49 19 32 36 31 47 42 48 25 44 14 48 37 43 3 6 1 27 47 48 45 25 48 46 44 20 18 13 1 21 35 48 46 40 32 5 43 35 43 9 2 10 4 38 19 46 8 11 40 38 16 26 31 30 28 36 22 44 33 28 49 42 13 40 20 12 23 4 34 39 15 46 40 6 33 47 5 29 21 11 35 49 29 18 17 23 42 3 16 43 41 41 44 


Warning message in readLines(con, n = chunksize):
"line 1164 appears to contain an embedded nul"


ERROR: Error in FUN(X[[i]], ...): sub'indice fuera de  los l'imites
